In [648]:
import numpy as np
import torch
import torch.nn as nn
import sklearn.datasets

### Data

In [649]:
x_train = np.random.rand(100, 5)
y_train = np.random.randint(0, 2, (100)).reshape(-1, 1)
x_train1 = torch.tensor(x_train)
y_train1 = torch.tensor(y_train)
# làm tròn
x_train = np.round(x_train, decimals=4)



### Hyperparameters

In [650]:
input_dim = 5
output_dim = 1
hidden_dim = 3
learning_rate = 0.001
epochs = 30

### Function

In [651]:
def sigmoid(x):
    return (1/(1+np.exp(-x)))

In [652]:
def get_item(NN):
    w1 = NN["w1"]
    b1 = NN["b1"]
    w2 = NN["w2"]
    b2 = NN["b2"]
    return w1, b1, w2, b2

### Weights & Bias

In [653]:
w1 = np.random.randn(input_dim, hidden_dim)
b1 = np.zeros((1, hidden_dim))
w2 = np.random.randn(hidden_dim, output_dim)
b2 = np.zeros((1, output_dim))
NN = {'w1':w1,'b1':b1,'w2':w2,'b2':b2}
NN2 = NN

## FOR AND BACK

In [654]:
def forward(x,NN):
    W1,b1,W2,b2=get_item(NN)
    z1 = x.dot(W1) + b1
    a1 = sigmoid(z1)
    z2 = a1.dot(W2) + b2
    a2 = sigmoid(z2)
    return a1,a2

In [655]:
def lossMSE(a2,y):

    return np.mean(np.square(a2 - y))

In [656]:
def backward(x, y,NN,learning_rate):

    W1, b1, W2, b2 = get_item(NN)
    a1,a2= forward(x,NN)

    # tính delta
    delta3 = a2 - y

    dW2 = np.dot(a1.T, delta3)
    db2 = np.sum(delta3, axis=0, keepdims=True)

    delta2 = np.dot(delta3, W2.T) * a1 * (1 - a1)
    dW1 = np.dot(x.T, delta2)
    db1 = np.sum(delta2, axis=0, keepdims=True)

    # update
    W1 += -learning_rate * dW1
    b1 += -learning_rate * db1
    W2 += -learning_rate * dW2
    b2 += -learning_rate * db2
    NN["w1"] = W1
    NN["b1"] = b1
    NN["w2"] = W2
    NN["b2"] = b2

    return lossMSE(a2,y)

## CODE CHECK

In [657]:
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size,NN2):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()
        W1, b1, W2, b2 = get_item(NN2)
        self.fc1.weight.data = torch.from_numpy(W1.T)
        self.fc1.bias.data = torch.from_numpy(b1)
        self.fc2.weight.data = torch.from_numpy(W2.T)
        self.fc2.bias.data = torch.from_numpy(b2)

    def forward(self, x):
        x = self.fc1(x)
        h = self.sigmoid(x)
        h = self.fc2(h)
        h = self.sigmoid(h)
        return h

In [658]:
model = SimpleNN(input_dim, hidden_dim, output_dim,NN2)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

### Compare

In [659]:

for i in range(epochs):
    print('EPOCH :',i+1)
    # hand
    loss_1 = backward(x_train, y_train, NN, learning_rate)
    print('hand:',loss_1)  

    # check
    out= model(x_train1)
    loss = nn.MSELoss()
    loss_2 = loss(out, y_train1.double())
    optimizer.zero_grad()
    loss_2.backward()
    optimizer.step()
    print('check :',loss_2.item())

EPOCH : 1
hand: 0.25049922357817106
check : 0.25035271457166425
EPOCH : 2
hand: 0.250352138063447
check : 0.2502222475810939
EPOCH : 3
hand: 0.250221747031442
check : 0.2501065566567643
EPOCH : 4
hand: 0.2501061234096367
check : 0.2500039332644649
EPOCH : 5
hand: 0.2500035596368782
check : 0.24991286414661296
EPOCH : 6
hand: 0.24991254332347165
check : 0.24983200952162232
EPOCH : 7
hand: 0.24983173546095577
check : 0.24976018360072705
EPOCH : 8
hand: 0.24975995094852724
check : 0.249696337202386
EPOCH : 9
hand: 0.24969614121624584
check : 0.2496395422589952
EPOCH : 10
hand: 0.24963937873977163
check : 0.24958897802590665
EPOCH : 11
hand: 0.24958884325666875
check : 0.24954391881811386
EPOCH : 12
hand: 0.24954380950967983
check : 0.24950372311501406
EPOCH : 13
hand: 0.24950363635742814
check : 0.24946782388811242
EPOCH : 14
hand: 0.2494677571074617
check : 0.24943572002021308
EPOCH : 15
hand: 0.24943567094023458
check : 0.24940696869744705
EPOCH : 16
hand: 0.2494069353054228
check : 0.2